In [781]:
# Importação das bibliotecas necessárias

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from nltk.util import ngrams
import nltk, re, string, collections
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, f1_score, recall_score

In [805]:
## Lendo e filtrando os dados:

txt = ""
with open("C:\\Users\\dougl\\Documents\\UNB\MDS\\2022-2-Squad03\\data\\raw\\mashing.txt", "r", encoding="utf-8") as g:
    txt = g.read()

mashings = txt.split("\n")

for i in range(len(mashings)):
    mashings[i] = mashings[i].strip()

texto = ""
with open("C:\\Users\\dougl\\Documents\\UNB\MDS\\2022-2-Squad03\\data\\raw\\large-2014.txt", "r", encoding="utf-8") as k:
    texto = k.read()

naturals = texto.split(" ")

for i in range(len(naturals)):
    naturals[i] = naturals[i].strip()

#Função para remoção de valores númericos e de pontuação das palavras "comuns" da base de dados      
naturals = list(filter(lambda token: token not in string.punctuation and token.isalpha(), naturals))

#Definição de duas séries de dados, uma com as palavras que são mashing e outra com as palavras naturais.
# Ambas para serem transformadas em ngramas posteriormente
X0 = pd.Series(naturals)
X1 = pd.Series(mashings)

In [807]:
#Separação da base de dados em treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=777)

#Diferentes algoritmos para vetorização dos ngramas, para usar um só comentar os demais
vectorizer = TfidfVectorizer(
                            lowercase=True,
                            analyzer='char',
                            binary=True,
                            strip_accents="unicode")

# vectorizer = CountVectorizer (lowercase = True,
#                               analyzer = 'char_wb',
#                               binary = False,
#                               strip_accents = "unicode")

# vectorizer = HashingVectorizer (
#                               lowercase = True,
#                               analyzer = 'char',
#                               binary = True,
#                               strip_accents = "unicode")

#Os índices dos dados de teste são resetados para a contagem começar a partir do 0
X_test = X_test.reset_index()['strings']
Y_test = Y_test.reset_index()['mashing']

#Os ngramas são propriamente vetorizados para poderem ser alimentados ao algoritmo
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [806]:
#Percorre as palavras naturais e transforma em ngramas
natbigrams = []
for row in range (len(X0)):
    ngram = map(''.join,list(everygrams(X0.iloc[row], 2, 4)))
    natbigrams.extend(ngram)

#Esse print comentado mostra todos os bigramas naturais
# print(natbigrams)

#Contagem da frequência de cada ngrama, printar essa variável mostra esses valores
natf = collections.Counter(natbigrams)

#Percorre os mashings e transforma em ngramas
mashbigrams = []
for row in range (len(X1)):
    ngram = map(''.join,list(everygrams(X1.iloc[row], 2, 4)))
    mashbigrams.extend(ngram)

#Esse print comentado mostra todos os bigramas de mashing
# print(mashbigrams)

#Diminui os dados de mashing para ficarem no mesmo tamanho que os de linguagem natural 
mashbigrams = mashbigrams[:len(natbigrams)]

#Contagem da frequência de cada ngrama, printar essa variável mostra esses valores
mashf = collections.Counter(mashbigrams)

#Cria um novo df somente com ngramas e separa as features (X) e os targets (Y)
mashdf = pd.DataFrame({'strings':mashbigrams + natbigrams, 'mashing': [1] * len(mashbigrams) + [0] * len(natbigrams)})
X = mashdf['strings']
Y = mashdf['mashing']

99276

In [808]:
## Treinando o modelo (Diferentes modelos estão comentados, para testar só descomentar um e comentar os outros):

# model = LogisticRegression()

# model = DecisionTreeClassifier()

# model = ExtraTreeClassifier()

# model = ExtraTreesClassifier()

model = RandomForestClassifier()

model.fit(X_train,Y_train)

RandomForestClassifier()

In [809]:
#Diferentes métricas para avalização da eficácia do modelo com a base de dados usada pra teste

Y_pred = model.predict(X_test)

acc = accuracy_score(Y_test, Y_pred) ##0.985
print('Acc:', acc)

score1 = balanced_accuracy_score(Y_test, Y_pred)

print('Acc:', score1)

score2 = f1_score(Y_test, Y_pred, labels=None, pos_label=0, average='binary', sample_weight=None, zero_division='warn')

print('Acc:', score2)

score3 = recall_score(Y_test, Y_pred, labels=None, pos_label=0, average='binary', sample_weight=None, zero_division='warn')

print('Acc:', score3)

Acc: 0.8785757453666398
Acc: 0.8784635629091233
Acc: 0.8834984295723605
Acc: 0.918240257131378


In [825]:
# Input para o algoritmo advinhar se é ks ou não:
input_data = [(input(""))]

#Processo de extração dos ngramas e tokenização do input
input_ngrams = []
for i in range (len(input_data)):
    ngram = map(''.join,list(everygrams(input_data[i], 2, 4)))
    input_ngrams.extend(ngram)

#Entrega das features ao modelo para ele adivinhar qual a classe se encaixaria melhor (1 = mashing)
#Para cada ngrama da palavra o algoritmo vai emitir uma classificação, uma mesma palavra pode ter ngramas de keyboard smashing e de palavras "normais"
pred = model.predict(vectorizer.transform(input_ngrams))

#É calculada a média da chance de ser keyboard smashing de todos os ngramas da palavra
predprob = model.predict_proba(vectorizer.transform(input_ngrams))[:,1]
prob = sum(predprob)/len(input_ngrams) * 100

#Esse print comentado mostra todos os ngramas formados da palavra e a probabilidade de ks de cada um deles
# print(input_ngrams,'\n',pred,'\n', prob)

# Se a probabilidade for maior que 50% a palavra é considera ks e a probabilidade é mostrada
if (prob >= 50):
    print (1)
    print(f"{prob:.3f}%")
else:
    print (0)
    print(f"{prob:.3f}%")


1
96.727%
